### Chemical Property Prediction and Evaluation

This notebook is *designed to perform chemical property prediction* using a *pre-trained model*, evaluate the model's performance using various metrics, and visualize the ROC curve to assess its discriminatory power. The specific chemical property being predicted is 'CYP1A2-inhibitor', and the code is organized into sections for clarity.

In [1]:
# Installing chemprop (https://github.com/chemprop/chemprop) for chemical property prediction
!pip install chemprop==2.0.0

In [2]:
# Importing necessary modules
import pandas as pd
import numpy as np
import os
import torch
import pandas as pd
import numpy as np
from chemprop import data, featurizers, models
from lightning import pytorch as pl
import math
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [3]:
# Define a dictionary for model paths
model_paths = {
    'caco2': os.path.join('Models', 'Absorption', 'Caco2.ckpt'),
    'solubility': os.path.join('Models', 'Absorption', 'Solubility.ckpt'),
    'lipophilicity': os.path.join('Models', 'Absorption', 'Lipophilicity.ckpt'),
    'ppbr': os.path.join('Models', 'Distribution', 'PPBR.ckpt'),
    'vdss': os.path.join('Models', 'Distribution', 'VDss.ckpt'),
    'cyp1a2-inhibitor': os.path.join('Models', 'Metabolism', 'CYP1A2-Inhibitor.ckpt'),
    'cyp1a2-substrate': os.path.join('Models', 'Metabolism', 'CYP1A2-Substrate.ckpt'),
    'cyp2c19-inhibitor': os.path.join('Models', 'Metabolism', 'CYP2C19-Inhibitor.ckpt'),
    'cyp2c19-substrate': os.path.join('Models', 'Metabolism', 'CYP2C19-Substrate.ckpt'),
    'cyp2c9-inhibitor': os.path.join('Models', 'Metabolism', 'CYP2C9-Inhibitor.ckpt'),
    'cyp2c9-substrate': os.path.join('Models', 'Metabolism', 'CYP2C9-Substrate.ckpt'),
    'cyp2d6-inhibitor': os.path.join('Models', 'Metabolism', 'CYP2D6-Inhibitor.ckpt'),
    'cyp2d6-substrate': os.path.join('Models', 'Metabolism', 'CYP2D6-Substrate.ckpt'),
    'cl-hepa': os.path.join('Models', 'Excretion', 'CL-Hepa.ckpt'),
    'cl-micro': os.path.join('Models', 'Excretion', 'CL-Micro.ckpt'),
    'half-life': os.path.join('Models', 'Excretion', 'Half-Life.ckpt'),
}

In [4]:
# Utility functions for loading and applying the model
def get_model_path(property_name):
    return model_paths.get(property_name.lower())

def load_model(checkpoint_path):
    return models.MPNN.load_from_checkpoint(checkpoint_path)

def featurize_smiles(smiles_list):
    test_data = [data.MoleculeDatapoint.from_smi(smi) for smi in smiles_list]
    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
    test_dset = data.MoleculeDataset(test_data, featurizer=featurizer)
    return data.build_dataloader(test_dset, shuffle=False)

def predict_property(model, dataloader):
    with torch.inference_mode():
        trainer = pl.Trainer(
            logger=False,
            enable_progress_bar=False,
            accelerator="cpu",
            devices=1
        )
        test_preds = trainer.predict(model, dataloader)
        test_preds_flat = [item for sublist in test_preds for item in sublist]
    return test_preds_flat

In [5]:
# Applying the trained model to make predictions for a specific chemical property
property_name = 'cyp1a2-inhibitor'
smiles_col = 'smiles'
dataset_path = os.path.join('Curated Datasets', 'Metabolism', 'CYP1A2-Inhibitor.csv')
smiles_list = pd.read_csv(dataset_path)[smiles_col].tolist()
model_path = get_model_path(property_name)
mpnn = load_model(model_path)
test_loader = featurize_smiles(smiles_list)
predictions = np.array(predict_property(mpnn, test_loader), dtype=object)
predictions

/opt/anaconda3/lib/python3.12/site-packages/chemprop/models/model.py:225: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hparams = torch.load(checkpoint_path)["hyper_paramete

array([[0.0],
       [0.0],
       [1.0],
       ...,
       [0.0],
       [0.0],
       [0.0]], dtype=object)

In [6]:
true = pd.read_csv(dataset_path)
true.head(10)

,smiles,Activity
0,CCCC(=O)Nc1ccc(N2CCN(CC)CC2)c(Cl)c1,0
1,O=c1[nH]c2cc3c(cc2cc1CN(CCCO)Cc1nnnn1Cc1ccc(F)...,1
2,CCN1C(=O)/C(=C2\SC(=S)N(CCCOC)C2=O)c2ccccc21,1
3,CC(=O)N(c1ccc2oc(=O)sc2c1)S(=O)(=O)c1cccs1,1
4,Clc1ccccc1-c1nc(-c2ccccc2)n[nH]1,1
5,COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1,1
6,N=c1n(CCN2CCOCC2)c2ccccc2n1CC(=O)c1ccc(Cl)c(Cl)c1,1
7,COc1ccc(/C(O)=C2/C(=O)C(=O)N(CCCC(=O)O)C2c2ccc...,0
8,COc1ccc(Nc2nc(N)nc(CSc3n[nH]c(-c4ccccc4)n3)n2)cc1,1
9,CCN(CC)C(=O)CSc1nnc(-c2cc3ccccc3cc2O)n1CC,1


In [7]:
"""Defining utility functions for metrics calculation"""

# Calculate the mean of activity values from the true dataset
mean = true['Activity'].mean()

# Function to determine whether a value should be rounded up (1) or down (0) based on the mean
def ceil(number):
    if number < mean:
        return 0
    else:
        return 1

# Function to convert a number to an integer
def toInt(number):
    return int(number)

"""Metrics calculation"""

# Mapping the 'ceil' function to prediction activity values and converting to NumPy arrays
pred_labels = np.asarray(list(map(ceil, predictions.tolist())))

# Mapping the 'toInt' function to true activity values and converting to NumPy arrays
true_labels = np.asarray(list(map(toInt, true['Activity'].tolist())))

# Calculating true positive, true negative, false positive, and false negative counts
tp = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
tn = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
fp = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
fn = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))

# Calculating specificity, sensitivity, accuracy, and balanced accuracy
specificity = tn / (fp + tn)
sensitivity = tp / (fn + tp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
balanced_accuracy = (specificity + sensitivity) / 2

# Displaying the calculated metrics
print(f"True Positives: {tp}")  # True Positives
print(f"True Negatives: {tn}")  # True Negatives
print(f"False Positives: {fp}")  # False Positives
print(f"False Negatives: {fn}")  # False Negatives
print(f"Specificity: {specificity}")  # Specificity
print(f"Sensitivity: {sensitivity}")  # Sensitivity
print(f"Accuracy: {accuracy}")  # Accuracy
print(f"Balanced accuracy: {balanced_accuracy}")  # Balanced Accuracy

True Positives: 4218
True Negatives: 6882
False Positives: 321
False Negatives: 1767
Specificity: 0.9554352353186173
Sensitivity: 0.7047619047619048
Accuracy: 0.8416742493175614
Balanced accuracy: 0.830098570040261


In [8]:
# This script provides a command-line interface for predicting molecular properties
def main():
    while True:
        property_name = input("Enter the property to calculate (e.g., solubility) or 'exit' to quit: ").strip().lower()
        
        if property_name == 'exit':
            print("Exiting the interface.")
            break
        
        model_path = get_model_path(property_name)
        
        if not model_path or not os.path.exists(model_path):
            print(f"Model for property '{property_name}' not found.")
            continue
        
        smiles_input = input("Enter the SMILES list (comma-separated): ").strip()
        smiles_list = [smi.strip() for smi in smiles_input.split(',')]
        
        mpnn = load_model(model_path)
        test_loader = featurize_smiles(smiles_list)
        
        test_preds = predict_property(mpnn, test_loader)
        
        test_preds_list = [pred[0] for pred in test_preds]
        
        df = pd.DataFrame({
            'SMILES': smiles_list,
            'Prediction': test_preds_list
        })
        
        print("Predictions:")
        print(df)

if __name__ == "__main__":
    main()

Enter the property to calculate (e.g., solubility) or 'exit' to quit:  vdss
Enter the SMILES list (comma-separated):  CC, CCCC


/opt/anaconda3/lib/python3.12/site-packages/chemprop/models/model.py:225: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hparams = torch.load(checkpoint_path)["hyper_paramete

Predictions:
  SMILES        Prediction
0     CC  tensor(-16.0948)
1   CCCC   tensor(-5.3520)


Enter the property to calculate (e.g., solubility) or 'exit' to quit:  exit


Exiting the interface.
